<a href="https://colab.research.google.com/github/tsaijoy/JoyTsai/blob/main/20250306.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1.   讀取試算表的內容
2.   內容要透過gemini api進行摘要
3.   將摘要寫入試算表的新分頁



授權雲端存取權限，等待接著要讀取試算表的內容

In [1]:
from google.colab import auth #對google sheet做授權
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

資料來源:https://docs.google.com/spreadsheets/d/1MVKjpjnGR-N9RbR5xA_mXQUgDsYCcyMJY9FL8s7UYfk/edit?usp=sharing

In [2]:
import pandas as pd #控制表單
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MVKjpjnGR-N9RbR5xA_mXQUgDsYCcyMJY9FL8s7UYfk/edit?usp=sharing') #記得換成你要放資料的那個試算表 #阿記得開共用

In [5]:
from gspread_dataframe import set_with_dataframe

# 選擇要更新的工作表（選擇第一個工作表）要讀第一個工作表
worksheet = gsheets.get_worksheet(0)  # 如果是其他工作表，可以用 get_worksheet(index) #(0)是第一個工作表 (1)是第二個工作表



In [6]:
worksheet

<Worksheet '工作表1' id:0>

In [7]:
#取擷取儲存格內的內容 #以陣列形式output
data = worksheet.get_all_values() #會以list的方式回傳 2D陣列

#將資料轉成DataFrame
df = pd.DataFrame(data) #從list轉換成df的形式

In [8]:
df

,0
0,Item
1,grains
2,proteins


https://ai.google.dev/

https://ai.google.dev/gemini-api/docs/quickstart?lang=python

In [9]:
test = '我是待處理的變數內容'

In [22]:
prompt = f'我想請你把以下內容進行摘要"{df.astype(str)}"並且提供給要減肥的人一些飲食建議'

In [23]:
prompt

'我想請你把以下內容進行摘要"          0\n0      Item\n1    grains\n2  proteins"並且提供給要減肥的人一些飲食建議'

In [24]:
from google.colab import userdata
apikey = userdata.get('key')

In [25]:
import google.generativeai as genai
from google.colab import userdata # Added import for userdata

# Retrieve apikey using userdata
apikey = userdata.get('key') # moved here before using apikey

genai.configure(api_key=apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

# Use the prompt variable instead of the string "prompt"
response = model.generate_content(prompt)
print(response.text)

摘要：提供的清單包含穀物和蛋白質兩種食物類別，這是健康飲食的重要組成部分。

減肥飲食建議：

為了減肥，你需要創造一個熱量赤字（消耗的熱量多於攝取的熱量）。這可以通過以下方式達成：

* **控制份量：**  即使是健康的食物，攝取過多也會導致體重增加。  學習正確的份量控制非常重要。  使用量杯和食物秤可以幫助你準確控制。

* **選擇全穀物：**  比起精製穀物（如白米、白麵包），選擇全穀物（如糙米、燕麥、全麥麵包）可以提供更多纖維，讓你更有飽腹感，幫助你減少整體熱量攝取。  清單中的“grains”應選擇全穀物。

* **攝取足夠的蛋白質：** 蛋白質能幫助你維持肌肉質量，增加飽腹感，並在減肥過程中保持能量。清單中的“proteins”應該包含瘦肉蛋白，例如雞胸肉、魚類、豆類、豆腐等。

* **多攝取蔬菜和水果：**  蔬菜和水果富含纖維和營養，熱量相對較低，能幫助你增加飽腹感，並提供身體所需的維生素和礦物質。

* **減少加工食品、含糖飲料和不健康的脂肪：** 這些食物通常熱量高、營養價值低，會阻礙你的減肥進程。

* **規律運動：**  運動能幫助你消耗更多卡路里，並提高新陳代謝。

* **多喝水：** 水能幫助你增加飽腹感，並促進新陳代謝。

**重要提示：**  減肥是一個循序漸進的過程，不要追求速效。  制定一個可持續的健康飲食計劃，並結合規律的運動，才能有效地減肥並維持健康體重。  如有需要，請諮詢註冊營養師或醫生，以獲得個人化的飲食建議。



In [28]:
type(response.text)

str

In [72]:
import pandas as pd
from io import StringIO

# 使用 StringIO 将字符串转为文件对象，并读取为 DataFrame
df_all = pd.read_csv(StringIO(response.text))

In [73]:
df_all

,摘要：提供的清單包含穀物和蛋白質兩種食物類別，這是健康飲食的重要組成部分。
0,減肥飲食建議：
1,為了減肥，你需要創造一個熱量赤字（消耗的熱量多於攝取的熱量）。這可以通過以下方式達成：
2,* **控制份量：** 即使是健康的食物，攝取過多也會導致體重增加。 學習正確的份量控制...
3,* **選擇全穀物：** 比起精製穀物（如白米、白麵包），選擇全穀物（如糙米、燕麥、全麥麵...
4,* **攝取足夠的蛋白質：** 蛋白質能幫助你維持肌肉質量，增加飽腹感，並在減肥過程中保持能...
5,* **多攝取蔬菜和水果：** 蔬菜和水果富含纖維和營養，熱量相對較低，能幫助你增加飽腹感...
6,* **減少加工食品、含糖飲料和不健康的脂肪：** 這些食物通常熱量高、營養價值低，會阻礙你...
7,* **規律運動：** 運動能幫助你消耗更多卡路里，並提高新陳代謝。
8,* **多喝水：** 水能幫助你增加飽腹感，並促進新陳代謝。
9,**重要提示：** 減肥是一個循序漸進的過程，不要追求速效。 制定一個可持續的健康飲食計...


In [77]:
# 選擇要更新的工作表（選擇第四個工作表）要讀第四個工作表
worksheet_allcommand = gsheets.get_worksheet(3)  # 如果是其他工作表，可以用 get_worksheet(index) #(0)是第一個工作表 (3)是第四個工作表
# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet_allcommand, df_all) #df是個變數(dataframe的形式)


In [ ]:
def translate_text(text, target_language):
    prompt = f"Translate the following text to {target_language}: {text}"
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def chat_bot():
    print("Welcome to the translation chatbot!")
    print("Type 'exit' to quit.")

    target_language = input("Please specify the target language (e.g., French, Spanish, Chinese): ")

    while True:
        user_input = input("Enter the text you want to translate: ")

        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        translation = translate_text(user_input, target_language)
        print(f"Translation to {target_language}: {translation}")

In [ ]:
chat_bot()

Welcome to the translation chatbot!
Type 'exit' to quit.
Please specify the target language (e.g., French, Spanish, Chinese): Chinese
Enter the text you want to translate: how are you
Translation to Chinese: 你好 (nǐ hǎo) 

Enter the text you want to translate: exit
Goodbye!
